# WordPiece Model(WPM)
WordPiece Model(WPM)은 구글에서 주로 기계 번역 및 음성 인식과 같은 자연어 처리(Natural Language Processing, NLP) 태스크에서 언어 모델의 성능을 향상시키기 위해 도입 되었으며. Byte Pair Encoder(BPE)의 변형 알고리즘으로, 주어진 텍스트 데이터에서 학습된 언어 모델의 우도(Likelihood)를 최대화하는 방식으로 토큰을 생성하는 알고리즘입니다.

WPM은 텍스트 내에서 우도가 가장 높은 쌍을 반복적으로 찾아내어 이를 병합하여 어휘를 동적으로 확장하며, 이러한 방식은 기계 번역 및 음성 인식과 같은 자연어 처리에서 OOV(Out-Of-Vocabulary) 문제를 해결하는데 도움을 준다.

✨ **논문**
https://static.googleusercontent.com/media/research.google.com/ko//pubs/archive/37842.pdf

---

위 논문에서 설명하고 있는 WPM의 훈련과정은 총 4가지로 나뉘어져 있습니다.

1. **단어 유닛 목록을 ASCII, 22,000개 일본어 11,000개 한국어를 유니코드 형태로 초기화**
    - 전세계의 모든 문자를 포함할 수 있는 Unicode 형식으로 단어 유닛 목록을 초기화
2. **1과정을 통해 만들어진 유닛 목록을 사용하여 학습 데이터에 대해 language model 구축**
    - 초기화된 단어 유닛 목록을 사용하여 훈련 데이터에 대한 언어 모델을 구축합니다. 여기서 언어 모델로는 N-gram 알고리즘 중에서 3~5-grams을 사용하며, Katz back-off language model을 활용

3. **모델에 결합한 2개의 단어 유닛을 추가했을 때 likelihood가 높은 pair를 새로운 단어 유닛으로 생성**
    - 구축된 언어 모델을 기반으로 두 개의 단어 유닛을 결합했을 때 likelihood가 가장 높은 pair를 새로운 단어 유닛으로 생성

4. **미리 결정한 단어 단위의 한계에 도달하거나 likelihood의 증가량이 특정 임계점보다 떨어질 때까지 해당 과정을 반복**
    - 특정한 단어 단위의 한계에 도달하거나, likelihood의 증가량이 특정 임계점보다 낮아질 때까지 생성된 단어 유닛을 계속해서 추가하고 언어 모델을 업데이트하여 훈련을 반복


**WordPiece Model(WPM)의 디코딩 개선과정**

초기에는 WPM이 길고 희귀한 쿼리에서 발생하는 문제를 처리하기 어려웠습니다. 이에 대한 해결책으로 디코딩 중에 해당 문제를 처리할 수 있도록 수정되었습니다.

---

## **디코딩 중의 문제 해결:**

1. **입력 데이터 처리:**
   - 원본 언어 모델 데이터에는 입력된 값 그대로 사용되며, WPM으로 LM 데이터를 분할할 때 공백이 보이면 각 단위의 앞뒤에 공백 마커(_, ~)를 붙이고, 그렇지 않으면 붙이지 않는다.

2. **공백 형태의 다양성:**
   - 왼쪽에만 공백이 있는 경우, 오른쪽에만 공백이 있는 경우, 양쪽에 공백이 있는 경우, 공백이 없는 경우 등 네 가지 형태로 나타납니다. 이때, 모든 조합이 존재하는 경우 단어의 크기가 원래 크기의 최대 4배까지 커질 수 있음

**디코딩 최적화:**

1. **LM과 사전 재구축:**
   - 위에서 정리한 인벤토리를 기반으로 언어 모델(LM)과 사전을 다시 구축

2. **디코딩 시 공백 삽입 위치 보존:**
   - 디코딩 시 모델에 따라 최적의 경로가 선택되어 공백을 넣을 위치와 넣지 않을 위치가 보존

**공백 디코딩 시나리오:**
1. **모든 공백 제거:**
   - " " -> ""

2. **이중 공백 마커를 공백으로 변경:**
   - "__" -> " "

3. **남은 단일 공백 제거:**
   - "_" -> ""

이러한 과정을 통해 WPM의 디코딩 과정이 최적화되어, 다양한 형태의 공백이 효과적으로 처리되고 최종 디코딩 결과가 보다 정교해지게 된다.

---

### 💻 예시<br>
아래 코퍼스에서 makers, over 는 모두 자주 이용되기 때문에 그 자체를 units 으로 이용하지만, Jet 은 자주 등장하지 않는 단어이기 때문에 subword units 인 ‘J’ 와 ‘et’ 으로 나눈다.

모든 단어의 시작에는 underbar, ‘_’ 를 붙입니다. 그렇다면 ‘Jet’ 은 ‘_Jet’ 이 되어 ‘_J et’ 으로 나뉘어집니다. makers 는 ‘_makers’ 가 됩니다.

```
Word : Jet makers feud over seat width with big orders at stake
Wordpieces: _J et _makers _fe ud _over _seat _width _with _big _orders _at _stake
```
Underbar("_") 는 문장 생성, 혹은 subwords 부터의 문장 복원을 위한 특수기호 입니다. Underbar 없이 subwords 를 띄어두면, 본래 띄어쓰기와 구분이 되지 않기 때문에 혼란을 줄 수 있어 Underbar를 이용아여 띄어쓰기를 구분한다.

문장을 복원하는 코드는 간단합니다. 띄어쓰기 기준으로 나눠진 tokens 을 concatenation 한 뒤, _로 다시 나눠 tokenize 하거나 _ 를 빈 칸으로 치환하여 문장으로 복원합니다.

```
def recover(tokens):
    sent = ''.join(tokens)
    sent = sent.replace('_', ' ')
    return sent
```



